In [2]:
%%configure -f
{
"conf":{
    "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.spark_catalog":"org.apache.iceberg.spark.SparkSessionCatalog",
    "spark.sql.catalog.spark_catalog.type":"hive",
    "spark.sql.catalog.dev":"org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.dev.type":"hadoop",
    "spark.sql.catalog.dev.warehouse":"s3://vasveena-test-demo/iceberg/catalog/tables/"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1643043567258_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1643043567258_0003,spark,idle,Link,Link,✔


In [4]:
val input_df = spark.read.parquet("s3://vasveena-test-demo/tmp/hudi-perf/input/")
input_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 5 more fields]
res6: Long = 2700000000


In [5]:
import org.apache.spark.sql.functions._

val input_df2=(input_df.withColumn("z", substring(md5(concat($"id")),1,1))
                       .withColumn("schema-v", lit("v1")).withColumn("data-v", lit("v2"))
                       .withColumn("trade_dt", substring($"modified_timestamp",1,10)))
input_df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
input_df2: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema-v|data-v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000000|    3|4000000|{E}|6e505939-f5fd-4ab...|2019|2021-04-02 00:05:02|  9|      v1|    v2|2021-04-02|
|4000001|    9|4000001|{F}|20486aca-2759-43f...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000002|   11|4000002|{G}|42962a21-a2dc-40d...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000003|    9|4000003|{H}|9841ad6d-1532-496...|2019|2021-04-02 00:05:02|  c|      v1|    v2|2021-04-02|
|4000004|    4|4000004|{I}|ff1a855a-cced-495...|2019|2021-04-02 00:05:02|  4|      v1|    v2|2021-04-02|
+-------+-----+-------+---+---

In [6]:
val input_df4 = input_df2.withColumnRenamed("schema-v", "schema_v").withColumnRenamed("data-v", "data_v")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df4: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]


In [10]:
spark.sql("drop table dev.db.iceberg_demo_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res12: org.apache.spark.sql.DataFrame = []


In [11]:
spark.sql("""CREATE TABLE dev.db.iceberg_demo_table (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_demo_table'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res13: org.apache.spark.sql.DataFrame = []


In [12]:
val t1 = System.nanoTime
input_df4.sortWithinPartitions("z","schema_v","data_v","trade_dt").writeTo("dev.db.iceberg_demo_table").overwritePartitions()
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 12988155581888
duration: String = 879.612711344seconds


In [13]:
spark.sql("select * from dev.db.iceberg_demo_table limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema_v|data_v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000016|    3|4000016|{U}|8cd50d7c-f23b-4b3...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000028|   11|4000028|{G}|b1768394-e790-40d...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000035|    3|4000035|{N}|d4e09d39-c604-45a...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000037|    3|4000037|{P}|8e3008c9-e455-4fb...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000063|   11|4000063|{P}|97ba4bca-b65b-465...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000070|    2|4000070|{W}|d918732f-6a06-46a...|2019|20

In [14]:
spark.sql("select * from dev.db.iceberg_demo_table where id = 4000012").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema_v|data_v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|20

In [17]:
spark.sql("""UPDATE dev.db.iceberg_demo_table
SET year = '2022', month = 1
WHERE id = 4000012
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res19: org.apache.spark.sql.DataFrame = []


In [18]:
spark.sql("select * from dev.db.iceberg_demo_table where id = 4000012").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema_v|data_v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000012|    1|4000012|{Q}|961f28a6-e8c9-426...|2022|20

In [25]:
spark.sql("select * from dev.db.iceberg_demo_table where id = 4000105").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema_v|data_v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000105|    3|4000105|{F}|e48ad35d-3966-4f9...|2019|20

In [26]:
spark.sql("DELETE FROM dev.db.iceberg_demo_table WHERE id = 4000105")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res28: org.apache.spark.sql.DataFrame = []


In [27]:
spark.sql("select * from dev.db.iceberg_demo_table where id = 4000105").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+---+---+----+----+------------------+---+--------+------+--------+
| id|month| sk|txt|uuid|year|modified_timestamp|  z|schema_v|data_v|trade_dt|
+---+-----+---+---+----+----+------------------+---+--------+------+--------+
+---+-----+---+---+----+----+------------------+---+--------+------+--------+



In [ ]:
spark.sql("SELECT * FROM "dev.db.iceberg_demo_table$iceberg_history"")

In [ ]:
spark.sql("SELECT * FROM dev.db.iceberg_demo_table where id = 4000105 FOR SYSTEM_TIME AS OF TIMESTAMP '2020-01-01 10:00:00'")

In [22]:
spark.sql("drop table dev.db.iceberg_demo_table_s3_shaping")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res24: org.apache.spark.sql.DataFrame = []


In [23]:
spark.sql("""CREATE TABLE dev.db.iceberg_demo_table_s3_shaping (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://vasveena-test-hmswh2/')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_demo_table_s3_shaping'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res25: org.apache.spark.sql.DataFrame = []


In [24]:
val t1 = System.nanoTime
input_df4.sortWithinPartitions("z","schema_v","data_v","trade_dt").writeTo("dev.db.iceberg_demo_table_s3_shaping").overwritePartitions()
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 14688608248215
duration: String = 852.279816876seconds


```
% aws s3 ls s3://vasveena-test-hmswh2/ | head 
                           PRE 000142ed/
                           PRE 000436c9/
                           PRE 00053095/
                           PRE 000b2446/
                           PRE 000b38ba/
                           PRE 00104f22/
                           PRE 00165274/
                           PRE 001df499/
                           PRE 0025e24f/
                           PRE 0026511a/


% aws s3 ls s3://vasveena-test-hmswh2/000b2446/db/iceberg_demo_table_s3_shaping/z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02/
2022-01-24 16:05:44    4916944 00344-3511-abdee369-8cde-4ae5-9a78-5d723c49e6dd-00013.parquet


% aws s3 ls s3://vasveena-test-hmswh2/000142ed/db/iceberg_demo_table_s3_shaping/z=2/schema_v=v1/data_v=v2/trade_dt=2021-04-02/
2022-01-24 16:02:53    6558022 00182-3349-938d8d66-f557-4795-84a5-93fdf99301fe-00003.parquet                          
                           
```

In [28]:
spark.sql("select * from dev.db.iceberg_demo_table_s3_shaping limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema_v|data_v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000012|    8|4000012|{Q}|961f28a6-e8c9-426...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000016|    3|4000016|{U}|8cd50d7c-f23b-4b3...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000028|   11|4000028|{G}|b1768394-e790-40d...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000035|    3|4000035|{N}|d4e09d39-c604-45a...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000037|    3|4000037|{P}|8e3008c9-e455-4fb...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000063|   11|4000063|{P}|97ba4bca-b65b-465...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|4000070|    2|4000070|{W}|d918732f-6a06-46a...|2019|20